In [10]:
import requests
import json
from rocrate.rocrate import ROCrate
from rocrate.model.person import Person
import pandas as pd

In [2]:
try:
    with open("/Users/annef/nird.json") as config:
        token = json.load(config)["token"]
except FileNotFoundError:
    print("config.json not found!")

In [3]:
url = "https://admin.ckan-archive-test.sigma2.no/api/3/action/package_search"
response = requests.get(url, headers={ "Authorization": token})

list_records = response.json()["result"]["results"]
list_records

[{'access_rights': 'public',
  'author': None,
  'author_email': None,
  'creator_user_id': '688f37a2-225d-41cb-8ee3-d8f81f54aa02',
  'cron': {'state': 'succeeded',
   'message': '',
   'submitted_date': '2024-12-10T12:00:26',
   'completed_date': '2024-12-10T12:00:39'},
  'doi': '10.82969/2024.35e87wkt',
  'id': '911363c8-9730-4c92-bb02-d76632e0df5a',
  'import_done': True,
  'import_source': 'project_area',
  'isopen': True,
  'language': 'en',
  'license_id': 'CC-BY-4.0',
  'license_title': 'Creative Commons Attribution 4.0',
  'license_url': 'https://creativecommons.org/licenses/by/4.0/',
  'maintainer': None,
  'maintainer_email': None,
  'metadata_created': '2024-12-10T10:42:12.781896',
  'metadata_modified': '2024-12-10T11:00:39.828459',
  'name': '11th-dataset-upload',
  'notes': 'Eleventh dataset upload',
  'num_resources': 2,
  'num_tags': 0,
  'organization': None,
  'owner_org': None,
  'private': False,
  'project': ['NS9999K'],
  'publisher': 'NIRD RDA',
  'release_date':

In [4]:
for rec in (x for x in list_records if x["doi"] == "10.82969/2024.sco9828x"):
    pass

In [6]:
crate = ROCrate()

## Add creators

In [5]:
print(rec["creators"])

[{'email': 'adilhasan2@gmail.com', 'first_name': 'Adil', 'last_name': 'Hasan', 'orcid': '0000-0003-2373-7112', 'organisation': 'Sigma2'}]


In [7]:
for creator in rec["creators"]:
    print(creator)
    orcid = creator.pop("orcid")
    print(orcid)
    crate.add(Person(crate, "https://orcid.org/" + orcid, properties=creator))

{'email': 'adilhasan2@gmail.com', 'first_name': 'Adil', 'last_name': 'Hasan', 'orcid': '0000-0003-2373-7112', 'organisation': 'Sigma2'}
0000-0003-2373-7112


## Add data

In [17]:
for toc in (x for x in rec["resources"] if x["name"] == "table_of_contents"):
    pass
toc["url"]

'https://admin.ckan-archive-test.sigma2.no/dataset/3c92e94c-33d7-4614-b229-78adc10924a8/download/table_of_contents_10.82969_2024.sco9828x.csv'

In [14]:
for resource in rec["resources"]:
    print(resource["name"])

table_of_contents
summary_contents


In [38]:
df = pd.read_csv(toc["url"], sep="|")
df = df.drop([0, 0])
df.head()

,filename,format,size,modified_date,fixity,http_url,s3_url
1,/archive/3c92e94c-33d7-4614-b229-78adc10924a8/...,application/octet-stream; charset=binary,5.120000e+08,2024-07-26T06:42:23.908413259 +0200,05f905f68d8db7fd838726b2743d0417,https://admin.ckan-archive-test.sigma2.no/dat...,https://s3.nird.sigma2.no/dataset/3c92e94c-33...
2,/archive/3c92e94c-33d7-4614-b229-78adc10924a8/...,application/octet-stream; charset=binary,5.120000e+08,2024-07-26T06:37:35.578637728 +0200,06cd31123e4b469b9d6eee57e18d557b,https://admin.ckan-archive-test.sigma2.no/dat...,https://s3.nird.sigma2.no/dataset/3c92e94c-33...
3,/archive/3c92e94c-33d7-4614-b229-78adc10924a8/...,application/octet-stream; charset=binary,1.963928e+09,2024-07-26T06:13:03.393304971 +0200,f9f07b9b149feace31c835a70258ece7,https://admin.ckan-archive-test.sigma2.no/dat...,https://s3.nird.sigma2.no/dataset/3c92e94c-33...
4,/archive/3c92e94c-33d7-4614-b229-78adc10924a8/...,application/octet-stream; charset=binary,1.024000e+09,2024-07-26T06:22:10.932604650 +0200,9e6ee7b8cdfe480a9edd65debe3fe7fd,https://admin.ckan-archive-test.sigma2.no/dat...,https://s3.nird.sigma2.no/dataset/3c92e94c-33...
5,/archive/3c92e94c-33d7-4614-b229-78adc10924a8/...,application/octet-stream; charset=binary,5.120000e+08,2024-07-26T06:47:20.666837550 +0200,cebec53585cf1a62069a6809218231cb,https://admin.ckan-archive-test.sigma2.no/dat...,https://s3.nird.sigma2.no/dataset/3c92e94c-33...


In [46]:
df.columns

Index(['filename ', ' format ', ' size ', ' modified_date ', ' fixity ',
       ' http_url ', ' s3_url'],
      dtype='object')

In [53]:
for row in df.itertuples(index=False, name=None):
    resource2add = {df.columns[i].strip() : row[i] for i, _ in enumerate(row)}
    url = resource2add.pop("http_url")
    print(url)
    print("properties = ", resource2add)
    resource = crate.add_file(url, fetch_remote = False, properties=resource2add)

 https://admin.ckan-archive-test.sigma2.no/dataset/3c92e94c-33d7-4614-b229-78adc10924a8/download/file5.dat 
properties =  {'filename': '/archive/3c92e94c-33d7-4614-b229-78adc10924a8/file5.dat ', 'format': ' application/octet-stream; charset=binary ', 'size': 512000000.0, 'modified_date': ' 2024-07-26T06:42:23.908413259 +0200 ', 'fixity': ' 05f905f68d8db7fd838726b2743d0417 ', 's3_url': ' https://s3.nird.sigma2.no/dataset/3c92e94c-33d7-4614-b229-78adc10924a8/download/file5.dat'}
 https://admin.ckan-archive-test.sigma2.no/dataset/3c92e94c-33d7-4614-b229-78adc10924a8/download/file4.dat 
properties =  {'filename': '/archive/3c92e94c-33d7-4614-b229-78adc10924a8/file4.dat ', 'format': ' application/octet-stream; charset=binary ', 'size': 512000000.0, 'modified_date': ' 2024-07-26T06:37:35.578637728 +0200 ', 'fixity': ' 06cd31123e4b469b9d6eee57e18d557b ', 's3_url': ' https://s3.nird.sigma2.no/dataset/3c92e94c-33d7-4614-b229-78adc10924a8/download/file4.dat'}
 https://admin.ckan-archive-test.sig

## Write to disk

In [54]:
crate.write("ro-crate")